## House Price Prediction With MLFLOW

In this tutorials we will 

- Run a hyperparameter tuning while training a model
- Log every Hyperparameter and metrics in the MLFLOW UI 

- Compare the results of the various runs in the MLflow UI

- Choose the best run and register it as a model

In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [ ]:
housing=fetch_california_housing()
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data['Price']=housing.target
from urllib.parse import urlparse
## Independent and Dependent features
X=data.drop(columns=["Price"])
y=data["Price"]

## Hyperparameter tuning using Grid Searchcv
def hyperparameter_tuning(X_train,y_train,param_grid):
    rf=RandomForestRegressor()
    grid_search=GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2,
                             scoring="neg_mean_squared_error")
    grid_search.fit(X_train,y_train)
    return grid_search

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [4]:
from mlflow.models import infer_signature
signature = infer_signature(X_train, y_train)

## Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# MLflow experiment
with mlflow.start_run():
    # Perform hyperparameter tuning
    grid_search = hyperparameter_tuning(X_train,y_train,param_grid)
    # Get the best model
    best_model = grid_search.best_estimator_
    
    # Evaluate the model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    # Log the best parameters and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_["n_estimators"])
    mlflow.log_param("best_max_depth", grid_search.best_params_["max_depth"])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_["min_samples_split"])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_["min_samples_leaf"])
    mlflow.log_param("mse", mse)
    
    # Tracking url
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
    
    if tracking_url_type_store !='file': # if mlflow tracking url is in the server, not local file based
        # Register the model
        mlflow.sklearn.log_model(best_model, "model", registered_model_name="Best Randomforest Model")
        print("11111111")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature=signature)
        print("2222")
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse}")   

Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/01/24 09:58:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RestException: RESOURCE_DOES_NOT_EXIST: Run '38e12a3c95dc4fb3a384fd2fe473120d' not found